In [1]:
import os 
import pandas as pd
import numpy as np
import pickle

In [2]:
home_string = ''
os.chdir(home_string)

In [8]:
from main.transport.fairness_calc import get_all_wasserstein, get_all_wasserstein_param

In [ ]:
for path_ in os.listdir('data/application/results/'):
    if 'oneto' in path_:
        with open(f'data/application/results/{path_}', 'rb') as con_:
            data_results = pickle.load(con_)
        
        s_vector = data_results['all_test_data'][:, data_results['data_out_sens']] == 1
        target_data = data_results['orig_data']
        predictions_oneto = data_results['data_oneto']

        dict_boost = get_all_wasserstein(predictions_oneto, s_vector, target_data)

        comb_dict = {'oneto': dict_boost}

        with open(f'data/application/aggregated_stats/stats_oneto_{path_}', 'wb') as con_:
            pickle.dump(comb_dict ,con_)
        
        

In [ ]:
for path_ in os.listdir('data/application/results/'):
    if not 'oneto' in path_:
        with open(f'data/application/results/{path_}', 'rb') as con_:
            data_results = pickle.load(con_)
        
        s_vector = data_results['all_test_data'][:, data_results['data_out_sens']] == 1
        target_data = data_results['orig_data']

        predictions_boost = data_results['preds_boosting_']
        predictions_lasso = data_results['preds_lasso_']

        dict_boost = get_all_wasserstein_param(predictions_boost, s_vector, target_data)
        dict_lasso = get_all_wasserstein_param(predictions_lasso, s_vector, target_data)

        comb_dict = {'boost': dict_boost, 
                     'lasso': dict_lasso}

        with open(f'data/application/aggregated_stats/param_stats_{path_}', 'wb') as con_:
            pickle.dump(comb_dict ,con_)
        

# Create Tables

Combine the distance inputs and error metrics

In [3]:
df_tmp = pd.DataFrame()
for file_ in os.listdir('data/application/aggregated_stats/'):
    if not 'oneto' in file_ and not 'secon_param_' in file_:
        with open(f'data/application/aggregated_stats/{file_}', 'rb') as con_:
            tmp_ = pickle.load(con_)

        df_tmp = pd.concat([df_tmp, pd.DataFrame(tmp_)])


data_performance = df_tmp.loc[['dist_param' not in v for v in df_tmp.index], ['boost', 'lasso']].reset_index(names='stat').groupby('stat').agg({'boost': ['mean', 'std'], 
                                                      'lasso': ['mean', 'std']})

df_tmp = pd.DataFrame()
for file_ in os.listdir('data/application/aggregated_stats/'):
    if 'secon_param' in file_:
        with open(f'data/application/aggregated_stats/{file_}', 'rb') as con_:
            tmp_ = pickle.load(con_)
            tmp_ = {'boost': {'dist_param_semifair': tmp_['boost']['dist_param_semifair'], 'dist_param_fair': tmp_['boost']['dist_param_fair']}, 
                    'lasso': {'dist_param_semifair': tmp_['lasso']['dist_param_semifair'], 'dist_param_fair': tmp_['lasso']['dist_param_fair']}, }

        df_tmp = pd.concat([df_tmp, pd.DataFrame(tmp_)])


data_distance_param = (df_tmp
                       .reset_index(names='stat')
                       .groupby('stat').agg({'boost': ['mean', 'std'], 
                                             'lasso': ['mean', 'std']}))

all_data_nononeto = pd.concat([data_distance_param, data_performance]).sort_values('stat')
all_data_nononeto

boost                  lasso           
                             mean        std        mean        std
stat                                                               
budget_fair             57.986819   0.132478   57.994371   0.126502
budget_param_fair       57.951468   0.142814   57.983878   0.125084
budget_param_semifair   58.718652   3.446411   58.605173   3.243557
budget_semifair         58.018477   0.112768   58.016347   0.111912
budget_unfair           58.050119   0.112162   58.038324   0.105427
dist_fair                0.191652   0.050814    0.264622   0.088228
dist_param_fair          0.143510   0.031008    0.147732   0.035204
dist_param_semifair      0.743986   0.055154    0.724579   0.054095
dist_semifair            0.851565   0.036952    0.843948   0.040881
dist_unfair              1.693537   0.060499    1.674267   0.056705
mse_1_fair               0.593504   0.015942    0.696857   0.016688
mse_1_param_fair         2.214214   0.939045    1.745076   0.523716
mse_1_param_semifair     0.791484   0.235619    0.774315   0.128111
mse_1_semifair           0.382939   0.008605    0.509696   0.009687
mse_1_unfair             0.311411   0.006549    0.448899   0.008514
mse_2_fair             469.553635   7.531519  477.257207   7.764375
mse_2_param_fair       623.881096  80.087173  617.279586  56.923829
mse_2_param_semifair   505.984944  21.625442  510.443642  17.112608
mse_2_semifair         467.779951   7.517180  475.498816   7.750224
mse_2_unfair           467.170616   7.524485  474.896897   7.692339

In [4]:
# Load "oneto" style predictions
df_tmp1 = pd.DataFrame()
for file_ in os.listdir('data/application/aggregated_stats/'):
    if 'oneto' in file_:
        with open(f'data/application/aggregated_stats/{file_}', 'rb') as con_:
            tmp_ = pickle.load(con_)
            tmp_ = tmp_['oneto']
            # only fair for oneto
            tmp_ = {'mse_1_fair': tmp_['mse_1_fair'], 
                    'mse_2_fair': tmp_['mse_2_fair'], 
                    'budget_fair': tmp_['budget_fair'], 
                    'dist_fair': tmp_['dist_fair']}

        df_tmp1 = pd.concat([df_tmp1, pd.DataFrame(tmp_, index=[0])])

benchmark_data = df_tmp1.melt(var_name='stat', value_name='oneto').groupby('stat').agg({'mean', 'std'})

In [5]:
merged_results = all_data_nononeto.merge(benchmark_data, on='stat', how='left')
merged_results.reset_index().to_csv('data/results_simulations.csv', index=False)
merged_results

boost                  lasso              
                             mean        std        mean        std   
stat                                                                  
budget_fair             57.986819   0.132478   57.994371   0.126502  \
budget_param_fair       57.951468   0.142814   57.983878   0.125084   
budget_param_semifair   58.718652   3.446411   58.605173   3.243557   
budget_semifair         58.018477   0.112768   58.016347   0.111912   
budget_unfair           58.050119   0.112162   58.038324   0.105427   
dist_fair                0.191652   0.050814    0.264622   0.088228   
dist_param_fair          0.143510   0.031008    0.147732   0.035204   
dist_param_semifair      0.743986   0.055154    0.724579   0.054095   
dist_semifair            0.851565   0.036952    0.843948   0.040881   
dist_unfair              1.693537   0.060499    1.674267   0.056705   
mse_1_fair               0.593504   0.015942    0.696857   0.016688   
mse_1_param_fair         2.214214   0.939045    1.745076   0.523716   
mse_1_param_semifair     0.791484   0.235619    0.774315   0.128111   
mse_1_semifair           0.382939   0.008605    0.509696   0.009687   
mse_1_unfair             0.311411   0.006549    0.448899   0.008514   
mse_2_fair             469.553635   7.531519  477.257207   7.764375   
mse_2_param_fair       623.881096  80.087173  617.279586  56.923829   
mse_2_param_semifair   505.984944  21.625442  510.443642  17.112608   
mse_2_semifair         467.779951   7.517180  475.498816   7.750224   
mse_2_unfair           467.170616   7.524485  474.896897   7.692339   

                            oneto             
                             mean        std  
stat                                          
budget_fair             58.588972   4.390242  
budget_param_fair             NaN        NaN  
budget_param_semifair         NaN        NaN  
budget_semifair               NaN        NaN  
budget_unfair                 NaN        NaN  
dist_fair                0.210665   0.077413  
dist_param_fair               NaN        NaN  
dist_param_semifair           NaN        NaN  
dist_semifair                 NaN        NaN  
dist_unfair                   NaN        NaN  
mse_1_fair               0.516007   0.048865  
mse_1_param_fair              NaN        NaN  
mse_1_param_semifair          NaN        NaN  
mse_1_semifair                NaN        NaN  
mse_1_unfair                  NaN        NaN  
mse_2_fair             498.864989  17.492403  
mse_2_param_fair              NaN        NaN  
mse_2_param_semifair          NaN        NaN  
mse_2_semifair                NaN        NaN  
mse_2_unfair                  NaN        NaN